# EV Sales Prediction - Week 1 Notebook
This notebook demonstrates dataset loading, basic EDA, preprocessing, and training a Random Forest model. Replace `data/ev_sales_sample.csv` with your downloaded Kaggle dataset (rename to `ev_sales.csv`).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

df = pd.read_csv('../data/ev_sales_sample.csv')
df.head()

In [ ]:
# Basic info
df.info()
df.describe(include='all')

In [ ]:
# Visualize sales by year
plt.figure(figsize=(8,4))
sns.barplot(x='Year', y='Sales', data=df)
plt.title('Sales by Year')
plt.show()

In [ ]:
# Preprocessing helper (similar to Streamlit app)
def preprocess(df):
    df2 = df.copy()
    colmap = {}
    for c in df2.columns:
        lc = c.lower()
        if 'price' in lc:
            colmap[c] = 'Price'
        if 'battery' in lc or 'kwh' in lc:
            colmap[c] = 'Battery_kWh'
        if 'range' in lc:
            colmap[c] = 'Range_km'
        if 'sales' in lc:
            colmap[c] = 'Sales'
        if 'year' in lc:
            colmap[c] = 'Year'
    df2 = df2.rename(columns=colmap)
    df2 = df2.dropna(subset=['Sales'])
    for col in ['Price','Battery_kWh','Range_km','Year']:
        if col in df2.columns:
            df2[col] = pd.to_numeric(df2[col], errors='coerce')
            df2[col] = df2[col].fillna(df2[col].mean())
    if 'Brand' in df2.columns:
        df2 = pd.get_dummies(df2, columns=['Brand'], drop_first=True)
    return df2

df_p = preprocess(df)
df_p.head()

In [ ]:
# Train a simple RandomForest model
X = df_p.drop(['Sales','Model'], axis=[0,0], errors='ignore')
y = df_p['Sales'].astype(float)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print('MSE:', mean_squared_error(y_test, preds))
print('R2:', r2_score(y_test, preds))
joblib.dump(model, '../app/model.pkl')
print('Saved model to app/model.pkl')